In [11]:
from sqlalchemy import create_engine
import pandas as pd

# Ejemplo de conexión 
def connect_to_database(user, password, host, db_name):
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}')
    return engine

In [12]:
def rentals_month(engine, month, year):
    query = f"""
        SELECT rental_id, customer_id, rental_date
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    # Ejecutar y devolver los datos como un DataFrame de pandas
    df = pd.read_sql(query, engine)
    return df

In [13]:
def rental_count_month(df, month, year):
    # Agrupar por customer_id y contar alquileres
    rental_counts = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_counts

In [14]:
def compare_rentals(df1, df2):
    # Fusionar los dos DataFrames en base a 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    
    # Añadir una columna de diferencia
    month1 = df1.columns[1]
    month2 = df2.columns[1]
    
    combined_df['difference'] = combined_df[month1] - combined_df[month2]
    
    return combined_df

In [15]:
# Conectar a la base de datos
engine = connect_to_database(user="root", password="root", host="127.0.0.1", db_name="sakila")

# Datos mayo y junio 2005
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Contar los alquileres para cada mes
rental_counts_may = rental_count_month(df_may, 5, 2005)
rental_counts_june = rental_count_month(df_june, 6, 2005)

# Comparar la actividad entre 2 meses
comparison = compare_rentals(rental_counts_may, rental_counts_june)

# Resultado
print(comparison)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0        -5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0        -2.0
3              4              0.0              6.0        -6.0
4              5              3.0              5.0        -2.0
..           ...              ...              ...         ...
593          595              1.0              2.0        -1.0
594          596              6.0              2.0         4.0
595          597              2.0              3.0        -1.0
596          598              0.0              1.0        -1.0
597          599              1.0              4.0        -3.0

[598 rows x 4 columns]
